In [12]:
import torch
from stealing_link.attack_eraser import StealingAttack
from stealing_link.partial_graph_generation import PartialGraphGeneration 
import config
import numpy as np
from torch_geometric.datasets import Planetoid, Coauthor
import torch_geometric.transforms as T
from sklearn.decomposition import PCA


file_path = '/IDEA/attack_materials/IDEA_cora_edge_0.05_SGC.pth'
model = torch.load(file_path)

args = {}
args['partial_graph_path'] = 'data/partial_graph_with_id/'
args['attack_partial_graph_ratio'] =0.5
args['attack_operator'] = 'concate_all'
args['attack_metric_type'] = 'kl_divergence'
args['dataset_name'] = 'cora'
args['ratio_unlearned'] = 0.05
args['exp'] = 'edge'

if args['dataset_name'] in ["cora", "pubmed", "citeseer"]:
            dataset = Planetoid(
                'raw_data', args['dataset_name'], transform=T.NormalizeFeatures()
            )
            labels = np.unique(dataset.data.y.numpy())
            data = dataset[0]
elif args['dataset_name'] in ["Coauthor_CS", "Coauthor_Phys"]:
    if args['dataset_name'] == "Coauthor_Phys":
        dataset = Coauthor(
            'raw_data', name="Physics",
        )
        data = dataset[0]
        feature = data.x
        pca = PCA(n_components=500)
        reduced_feature = pca.fit_transform(feature.cpu().detach().numpy())
        data.x = torch.from_numpy(reduced_feature)
    else:
        dataset = Coauthor(
            config.RAW_DATA_PATH, name="CS",
        )
        data = dataset[0]
else:
    raise Exception("unsupported dataset")




# PartialGraphGeneration(args, data, model['removed_edges'], model['predicted_prob'])
# StealingAttack(args)


In [4]:
model.keys()

dict_keys(['removed_edges', 'unlearned_feature_pre', 'predicted_prob', 'train_indices', 'test_indices'])

In [1]:
import pandas as pd
# pd.read_csv('result/attack_results.csv')
# Read the CSV file
df = pd.read_csv('result/attack_results.csv')

# Calculate the mean and standard deviation for accuracy

mean_accuracy = df['Test AUC'].mean()*100
std_accuracy = df['Test AUC'].std()*100
print(f'& xx.xx $\pm$ x.x     & {mean_accuracy:.2f} $\pm$ {std_accuracy:.1f}        \\\\')

& xx.xx $\pm$ x.x     & 50.11 $\pm$ 0.9        \\


Node attack

In [ ]:
import torch
from mia.MLG_TSTF import MIA
import config
import numpy as np
from torch_geometric.datasets import Planetoid, Coauthor
import torch_geometric.transforms as T
from sklearn.decomposition import PCA

file_path = '../IDEA/attack_materials/IDEA_cora_node_0.05_SGC.pth'
model = torch.load(file_path)

args = {}
args['dataset_name'] = 'cora'
args['ratio_unlearned'] = 0.05
args['exp'] = 'node'

if args['dataset_name'] in ["cora", "pubmed", "citeseer"]:
            dataset = Planetoid(
                'raw_data', args['dataset_name'], transform=T.NormalizeFeatures()
            )
            labels = np.unique(dataset.data.y.numpy())
            data = dataset[0]
elif args['dataset_name'] in ["Coauthor_CS", "Coauthor_Phys"]:
    if args['dataset_name'] == "Coauthor_Phys":
        dataset = Coauthor(
            'raw_data', name="Physics",
        )
        data = dataset[0]
        feature = data.x
        pca = PCA(n_components=500)
        reduced_feature = pca.fit_transform(feature.cpu().detach().numpy())
        data.x = torch.from_numpy(reduced_feature)
    else:
        dataset = Coauthor(
            config.RAW_DATA_PATH, name="CS",
        )
        data = dataset[0]
else:
    raise Exception("unsupported dataset")

train_indices = model['train_indices'].nonzero().view(-1)
test_indices = model['test_indices'].nonzero().view(-1)
MIA(args, model['predicted_prob'], model['removed_nodes'], train_indices, test_indices)


Posteior Residue

In [13]:
import torch
from stealing_link.attack_eraser import StealingAttack
from stealing_link.partial_graph_generation import PartialGraphGeneration 
import config
import numpy as np
from torch_geometric.datasets import Planetoid, Coauthor
import torch_geometric.transforms as T
from sklearn.decomposition import PCA
import torch.nn.functional as F

file_path = '../IDEA/attack_materials/IDEA_cora_partial_feature_0.05_SGC.pth'
model = torch.load(file_path)

# Convert data.y to one-hot encoding
one_hot_y = F.one_hot(data.y, num_classes=dataset.num_classes).cpu()




In [14]:
import torch
import torch.nn as nn

# Define your predicted probabilities and target labels
predicted_probs = (model['unlearned_feature_pre'])[model['unlearned_feature_node_idx']]  # Example predicted probabilities
target_labels = data.y[model['unlearned_feature_node_idx']]  # Example target labels

# Calculate the cross-entropy loss
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(predicted_probs, target_labels)

print(loss.item())


1.260506272315979


In [15]:
# Define your predicted probabilities and target labels
predicted_probs = (model['predicted_prob'].cpu())[model['unlearned_feature_node_idx']]  # Example predicted probabilities
target_labels = data.y[model['unlearned_feature_node_idx']]  # Example target labels

# Calculate the cross-entropy loss
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(predicted_probs, target_labels)

print(loss.item())

1.2976230382919312
